In [1]:
# def main(debug=False):
#     num_rows = 10000 if debug else None
#     with timer("train & test"):
#         df = train_test(num_rows)
#     with timer("historical transactions"):
#         df = pd.merge(df, historical_transactions(num_rows), on='card_id', how='outer')
#     with timer("new merchants"):
#         df = pd.merge(df, new_merchant_transactions(num_rows), on='card_id', how='outer')
#     with timer("additional features"):
#         df = additional_features(df)
#     with timer("split train & test"):
#         train_df = df[df['target'].notnull()]
#         test_df = df[df['target'].isnull()]
#         del df
#         gc.collect()
#     with timer("Run LightGBM with kfold"): 
#         kfold_lightgbm(train_df, test_df, num_folds=11, stratified=False, debug=debug)

# if __name__ == "__main__":
#     submission_file_name = "submission.csv"
#     with timer("Full model run"):
#         main(debug=False)

In [2]:
from contextlib import contextmanager
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import os
import time
import gc
import warnings

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold


warnings.simplefilter(action='ignore', category=Warning)
pd.options.mode.chained_assignment = None
# warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
# warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

### Step01 Preprocessing train & test

In [4]:
# load csv
num_rows=None
train_df = pd.read_csv('../data/train.csv', index_col=['card_id'], nrows=num_rows)
test_df = pd.read_csv('../data/test.csv', index_col=['card_id'], nrows=num_rows)

print("Train samples: {}, test samples: {}".format(len(train_df), len(test_df)))
display(train_df.head())
display(test_df.head())
train_df.describe().T

Train samples: 201917, test samples: 123623


,first_active_month,feature_1,feature_2,feature_3,target
card_id,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283
C_ID_3d0044924f,2017-01,4,1,0,0.392913
C_ID_d639edf6cd,2016-08,2,2,0,0.688056
C_ID_186d6a6901,2017-09,4,3,0,0.142495
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749


,first_active_month,feature_1,feature_2,feature_3
card_id,,,,
C_ID_0ab67a22ab,2017-04,3,3,1
C_ID_130fd0cbdd,2017-01,2,3,0
C_ID_b709037bc5,2017-08,5,1,1
C_ID_d27d835a9f,2017-12,2,1,0
C_ID_2b5e3df5c2,2015-12,5,1,1


,count,mean,std,min,25%,50%,75%,max
feature_1,201917.0,3.105311,1.186160,1.000000,2.00000,3.000000,4.000000,5.000000
feature_2,201917.0,1.745410,0.751362,1.000000,1.00000,2.000000,2.000000,3.000000
feature_3,201917.0,0.565569,0.495683,0.000000,0.00000,1.000000,1.000000,1.000000
target,201917.0,-0.393636,3.850500,-33.219281,-0.88311,-0.023437,0.765453,17.965068


In [5]:
# outlier
train_df['outliers'] = 0
train_df.loc[train_df['target'] < -30, 'outliers'] = 1

In [6]:
# set target as nan
test_df['target'] = np.nan

In [7]:
# append train + test vertically ---> df
df = pd.concat([train_df, test_df])
df.shape

(325540, 6)

In [8]:
del train_df, test_df
gc.collect()

0

In [9]:
# to datetime
df['first_active_month'] = pd.to_datetime(df['first_active_month'])

In [10]:
# to datetime
df['first_active_month'] = pd.to_datetime(df['first_active_month'])

# datetime features
df['quarter'] = df['first_active_month'].dt.quarter
df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days

df['days_feature1'] = df['elapsed_time'] * df['feature_1']
df['days_feature2'] = df['elapsed_time'] * df['feature_2']
df['days_feature3'] = df['elapsed_time'] * df['feature_3']

df['days_feature1_ratio'] = df['feature_1'] / df['elapsed_time']
df['days_feature2_ratio'] = df['feature_2'] / df['elapsed_time']
df['days_feature3_ratio'] = df['feature_3'] / df['elapsed_time']

In [11]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns = categorical_columns, dummy_na = nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

df, cols = one_hot_encoder(df, nan_as_category=False)
display(df.head())
print(cols)

,first_active_month,feature_1,feature_2,feature_3,target,outliers,quarter,elapsed_time,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio
card_id,,,,,,,,,,,,,,
C_ID_92a2005557,2017-06-01,5,2,1,-0.820283,0.0,2.0,2666.0,13330.0,5332.0,2666.0,0.001875,0.000750,0.000375
C_ID_3d0044924f,2017-01-01,4,1,0,0.392913,0.0,1.0,2817.0,11268.0,2817.0,0.0,0.001420,0.000355,0.000000
C_ID_d639edf6cd,2016-08-01,2,2,0,0.688056,0.0,3.0,2970.0,5940.0,5940.0,0.0,0.000673,0.000673,0.000000
C_ID_186d6a6901,2017-09-01,4,3,0,0.142495,0.0,3.0,2574.0,10296.0,7722.0,0.0,0.001554,0.001166,0.000000
C_ID_cdbd2c0db2,2017-11-01,1,3,0,-0.159749,0.0,4.0,2513.0,2513.0,7539.0,0.0,0.000398,0.001194,0.000000


[]


In [12]:
for f in ['feature_1','feature_2','feature_3']:
    order_label = df.groupby([f])['outliers'].mean()
    df[f] = df[f].map(order_label)

df.head()

,first_active_month,feature_1,feature_2,feature_3,target,outliers,quarter,elapsed_time,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio
card_id,,,,,,,,,,,,,,
C_ID_92a2005557,2017-06-01,0.013145,0.008752,0.011428,-0.820283,0.0,2.0,2666.0,13330.0,5332.0,2666.0,0.001875,0.000750,0.000375
C_ID_3d0044924f,2017-01-01,0.010712,0.011385,0.010283,0.392913,0.0,1.0,2817.0,11268.0,2817.0,0.0,0.001420,0.000355,0.000000
C_ID_d639edf6cd,2016-08-01,0.010610,0.008752,0.010283,0.688056,0.0,3.0,2970.0,5940.0,5940.0,0.0,0.000673,0.000673,0.000000
C_ID_186d6a6901,2017-09-01,0.010712,0.014166,0.010283,0.142495,0.0,3.0,2574.0,10296.0,7722.0,0.0,0.001554,0.001166,0.000000
C_ID_cdbd2c0db2,2017-11-01,0.008058,0.014166,0.010283,-0.159749,0.0,4.0,2513.0,2513.0,7539.0,0.0,0.000398,0.001194,0.000000


In [13]:
df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
df['feature_mean'] = df['feature_sum']/3
df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
df['feature_var'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

df.head()

,first_active_month,feature_1,feature_2,feature_3,target,outliers,quarter,elapsed_time,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_mean,feature_max,feature_min,feature_var
card_id,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,2017-06-01,0.013145,0.008752,0.011428,-0.820283,0.0,2.0,2666.0,13330.0,5332.0,2666.0,0.001875,0.000750,0.000375,0.033324,0.011108,0.013145,0.008752,0.002214
C_ID_3d0044924f,2017-01-01,0.010712,0.011385,0.010283,0.392913,0.0,1.0,2817.0,11268.0,2817.0,0.0,0.001420,0.000355,0.000000,0.032379,0.010793,0.011385,0.010283,0.000555
C_ID_d639edf6cd,2016-08-01,0.010610,0.008752,0.010283,0.688056,0.0,3.0,2970.0,5940.0,5940.0,0.0,0.000673,0.000673,0.000000,0.029645,0.009882,0.010610,0.008752,0.000992
C_ID_186d6a6901,2017-09-01,0.010712,0.014166,0.010283,0.142495,0.0,3.0,2574.0,10296.0,7722.0,0.0,0.001554,0.001166,0.000000,0.035161,0.011720,0.014166,0.010283,0.002129
C_ID_cdbd2c0db2,2017-11-01,0.008058,0.014166,0.010283,-0.159749,0.0,4.0,2513.0,2513.0,7539.0,0.0,0.000398,0.001194,0.000000,0.032508,0.010836,0.014166,0.008058,0.003091


### Step02 Preprocessing historical transactions

In [14]:
# load csv
nrows=None
hist_df = pd.read_csv('../data/historical_transactions.csv', nrows=num_rows)

hist_df.head().T

,0,1,2,3,4
authorized_flag,Y,Y,Y,Y,Y
card_id,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc
city_id,88,88,88,88,88
category_1,N,N,N,N,N
installments,0,0,0,0,0
category_3,A,A,A,A,A
merchant_category_id,80,367,80,560,80
merchant_id,M_ID_e020e9b302,M_ID_86ec983688,M_ID_979ed661fc,M_ID_e6d5ae8ea6,M_ID_e020e9b302
month_lag,-8,-7,-6,-5,-11
purchase_amount,-0.703331,-0.733128,-0.720386,-0.735352,-0.722865


In [15]:
hist_df.describe().T

,count,mean,std,min,25%,50%,75%,max
city_id,29112361.0,129.325617,104.256301,-1.000000,53.000000,90.000000,212.000000,3.470000e+02
installments,29112361.0,0.648495,2.795577,-1.000000,0.000000,0.000000,1.000000,9.990000e+02
merchant_category_id,29112361.0,481.013028,249.375714,-1.000000,307.000000,454.000000,705.000000,8.910000e+02
month_lag,29112361.0,-4.487294,3.588800,-13.000000,-7.000000,-4.000000,-2.000000,0.000000e+00
purchase_amount,29112361.0,0.036401,1123.521648,-0.746908,-0.720356,-0.688349,-0.603254,6.010604e+06
category_2,26459497.0,2.194578,1.531896,1.000000,1.000000,1.000000,3.000000,5.000000e+00
state_id,29112361.0,10.566790,6.366927,-1.000000,9.000000,9.000000,16.000000,2.400000e+01
subsector_id,29112361.0,26.848388,9.692793,-1.000000,19.000000,29.000000,34.000000,4.100000e+01


In [16]:
# fillna
hist_df['category_2'].fillna(1.0,inplace=True)
hist_df['category_3'].fillna('A',inplace=True)
hist_df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
hist_df['installments'].replace(-1, np.nan,inplace=True)
hist_df['installments'].replace(999, np.nan,inplace=True)

In [17]:
# trim
hist_df['purchase_amount'] = hist_df['purchase_amount'].apply(lambda x: min(x, 0.8))

In [18]:
# Y/N to 1/0
hist_df['authorized_flag'] = hist_df['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
hist_df['category_1'] = hist_df['category_1'].map({'Y': 1, 'N': 0}).astype(int)
hist_df['category_3'] = hist_df['category_3'].map({'A':0, 'B':1, 'C':2})

In [19]:
# datetime features
hist_df['purchase_date'] = pd.to_datetime(hist_df['purchase_date'])
hist_df['month'] = hist_df['purchase_date'].dt.month
hist_df['day'] = hist_df['purchase_date'].dt.day
hist_df['hour'] = hist_df['purchase_date'].dt.hour
hist_df['weekofyear'] = hist_df['purchase_date'].apply(lambda x: x.isocalendar()[1])
hist_df['weekday'] = hist_df['purchase_date'].dt.weekday
hist_df['weekend'] = (hist_df['purchase_date'].dt.weekday >=5).astype(int)

In [20]:
# additional features
hist_df['price'] = hist_df['purchase_amount'] / hist_df['installments']

In [21]:
#Christmas : December 25 2017
hist_df['Christmas_Day_2017']=(pd.to_datetime('2017-12-25')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
#Mothers Day: May 14 2017
hist_df['Mothers_Day_2017']=(pd.to_datetime('2017-06-04')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
#fathers day: August 13 2017
hist_df['fathers_day_2017']=(pd.to_datetime('2017-08-13')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
#Childrens day: October 12 2017
hist_df['Children_day_2017']=(pd.to_datetime('2017-10-12')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
#Valentine's Day : 12th June, 2017
hist_df['Valentine_Day_2017']=(pd.to_datetime('2017-06-12')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
#Black Friday : 24th November 2017
hist_df['Black_Friday_2017']=(pd.to_datetime('2017-11-24') - hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)

#2018
#Mothers Day: May 13 2018
hist_df['Mothers_Day_2018']=(pd.to_datetime('2018-05-13')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)

In [22]:
hist_df['month_diff'] = ((datetime.datetime.today() - hist_df['purchase_date']).dt.days)//30
hist_df['month_diff'] += hist_df['month_lag']

hist_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,weekend,price,Christmas_Day_2017,Mothers_Day_2017,fathers_day_2017,Children_day_2017,Valentine_Day_2017,Black_Friday_2017,Mothers_Day_2018,month_diff
0,1,C_ID_4e6213e9bc,88,0,0.0,0,80,M_ID_e020e9b302,-8,-0.703331,...,1,-inf,0,0,48,0,0,0,0,80
1,1,C_ID_4e6213e9bc,88,0,0.0,0,367,M_ID_86ec983688,-7,-0.733128,...,1,-inf,0,0,28,88,0,0,0,80
2,1,C_ID_4e6213e9bc,88,0,0.0,0,80,M_ID_979ed661fc,-6,-0.720386,...,0,-inf,0,0,3,63,0,0,0,80
3,1,C_ID_4e6213e9bc,88,0,0.0,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,...,1,-inf,0,0,0,39,0,82,0,80
4,1,C_ID_4e6213e9bc,88,0,0.0,0,80,M_ID_e020e9b302,-11,-0.722865,...,0,-inf,0,85,0,0,93,0,0,80


In [23]:
# additional features
hist_df['duration'] = hist_df['purchase_amount']*hist_df['month_diff']
hist_df['amount_month_ratio'] = hist_df['purchase_amount']/hist_df['month_diff']

In [24]:
# reduce memory usage
# reduce memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

hist_df = reduce_mem_usage(hist_df)

Memory usage after optimization is: 1832.41 MB
Decreased by 71.6%


In [25]:
# Derive aggregate features
col_unique =['subsector_id', 'merchant_id', 'merchant_category_id']
col_seas = ['month', 'hour', 'weekofyear', 'weekday', 'day']

aggs = {}
for col in col_unique:
    aggs[col] = ['nunique']

for col in col_seas:
    aggs[col] = ['nunique', 'mean', 'min', 'max']

aggs['purchase_amount'] = ['sum','max','min','mean','var','skew']
aggs['installments'] = ['sum','max','mean','var','skew']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var','skew']
aggs['month_diff'] = ['max','min','mean','var','skew']
aggs['authorized_flag'] = ['mean']
aggs['weekend'] = ['mean'] # overwrite
aggs['weekday'] = ['mean'] # overwrite
aggs['day'] = ['nunique', 'mean', 'min'] # overwrite
aggs['category_1'] = ['mean']
aggs['category_2'] = ['mean']
aggs['category_3'] = ['mean']
aggs['card_id'] = ['size','count']
aggs['price'] = ['sum','mean','max','min','var']
aggs['Christmas_Day_2017'] = ['mean']
aggs['Mothers_Day_2017'] = ['mean']
aggs['fathers_day_2017'] = ['mean']
aggs['Children_day_2017'] = ['mean']
aggs['Valentine_Day_2017'] = ['mean']
aggs['Black_Friday_2017'] = ['mean']
aggs['Mothers_Day_2018'] = ['mean']
aggs['duration']=['mean','min','max','var','skew']
aggs['amount_month_ratio']=['mean','min','max','var','skew']

hist_df['category_2'] = hist_df['category_2'].astype('int16')

for col in ['category_2','category_3']:
    hist_df[col+'_mean'] = hist_df.groupby([col])['purchase_amount'].transform('mean')
    hist_df[col+'_min'] = hist_df.groupby([col])['purchase_amount'].transform('min')
    hist_df[col+'_max'] = hist_df.groupby([col])['purchase_amount'].transform('max')
    hist_df[col+'_sum'] = hist_df.groupby([col])['purchase_amount'].transform('sum')
    aggs[col+'_mean'] = ['mean']

hist_df = hist_df.reset_index().groupby('card_id').agg(aggs)
hist_df.head().T

card_id                      C_ID_00007093c1 C_ID_0001238066 C_ID_0001506ef0  \
subsector_id         nunique              13              17              12   
merchant_id          nunique              29              65              28   
merchant_category_id nunique              18              29              19   
month                nunique              12               6              11   
                     mean           6.375839        7.308943         6.80303   
...                                      ...             ...             ...   
amount_month_ratio   max            0.010124        0.009598        0.010002   
                     var             0.00001        0.000006        0.000029   
                     skew           2.828021        4.054585        2.398635   
category_2_mean      mean          -0.593705       -0.606118       -0.591088   
category_3_mean      mean          -0.552033       -0.509526       -0.639323   

card_id                      C_ID_0001793786 C_ID_000183fdda  
subsector_id         nunique              24              21  
merchant_id          nunique             119              73  
merchant_category_id nunique              48              36  
month                nunique              10               7  
                     mean           6.671296        6.881944  
...                                      ...             ...  
amount_month_ratio   max            0.009636        0.010002  
                     var            0.000033        0.000021  
                     skew           1.292712        2.549857  
category_2_mean      mean          -0.601817       -0.591955  
category_3_mean      mean          -0.639181       -0.510981  

[73 rows x 5 columns]

In [26]:
hist_df.columns.tolist()[:10]

[('subsector_id', 'nunique'),
 ('merchant_id', 'nunique'),
 ('merchant_category_id', 'nunique'),
 ('month', 'nunique'),
 ('month', 'mean'),
 ('month', 'min'),
 ('month', 'max'),
 ('hour', 'nunique'),
 ('hour', 'mean'),
 ('hour', 'min')]

In [27]:
# change column name
hist_df.columns = pd.Index([e[0] + "_" + e[1] for e in hist_df.columns.tolist()])
hist_df.columns = ['hist_'+ c for c in hist_df.columns]
hist_df.head()

,hist_subsector_id_nunique,hist_merchant_id_nunique,hist_merchant_category_id_nunique,hist_month_nunique,hist_month_mean,hist_month_min,hist_month_max,hist_hour_nunique,hist_hour_mean,hist_hour_min,...,hist_duration_max,hist_duration_var,hist_duration_skew,hist_amount_month_ratio_mean,hist_amount_month_ratio_min,hist_amount_month_ratio_max,hist_amount_month_ratio_var,hist_amount_month_ratio_skew,hist_category_2_mean_mean,hist_category_3_mean_mean
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,13,29,18,12,6.375839,1,12,18,14.416107,0,...,63.1875,409.551880,2.794786,-0.006553,-0.009132,0.010124,0.000010,2.828021,-0.593705,-0.552033
C_ID_0001238066,17,65,29,6,7.308943,1,12,20,14.739837,0,...,61.4375,231.127884,4.055531,-0.007377,-0.009224,0.009598,0.000006,4.054585,-0.606118,-0.509526
C_ID_0001506ef0,12,28,19,11,6.803030,1,12,15,12.606061,0,...,64.0000,1168.780396,2.398956,-0.006709,-0.009361,0.010002,0.000029,2.398635,-0.591088,-0.639323
C_ID_0001793786,24,119,48,10,6.671296,1,10,21,15.606481,0,...,67.1875,1645.351929,1.295421,-0.003367,-0.008980,0.009636,0.000033,1.292712,-0.601817,-0.639181
C_ID_000183fdda,21,73,36,7,6.881944,1,12,19,16.465278,0,...,64.0000,856.859802,2.555445,-0.006436,-0.009224,0.010002,0.000021,2.549857,-0.591955,-0.510981


In [28]:
hist_df['hist_purchase_date_diff'] = (hist_df['hist_purchase_date_max']-hist_df['hist_purchase_date_min']).dt.days
hist_df['hist_purchase_date_average'] = hist_df['hist_purchase_date_diff']/hist_df['hist_card_id_size']
hist_df['hist_purchase_date_uptonow'] = (datetime.datetime.today()-hist_df['hist_purchase_date_max']).dt.days
hist_df['hist_purchase_date_uptomin'] = (datetime.datetime.today()-hist_df['hist_purchase_date_min']).dt.days

# reduce memory usage
hist_df = reduce_mem_usage(hist_df)

Memory usage after optimization is: 56.19 MB
Decreased by 60.4%
